In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(30)

from gevent.lock import BoundedSemaphore
semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

import os
import shutil
directory_out = './file/2.part_number'
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)
import re

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

from urllib.parse import urlparse, parse_qs, urlencode

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = =  = = =  = = =

print('Data Loading...')
list_series_error, df_input = [], pd.read_excel('./file/1.page.xlsx',
                                                header=0,
                                                dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)
df_input['SKU Count'] = df_input['SKU Count'].astype(int)
df_input['Page'] = df_input['Page'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

# = = =  = = =  = = =

def crawler(crawler_tuple):
    global list_series_error, crawler_remain

    crawler_index, crawler_series = crawler_tuple

    # = = =  = = =  = = =

    try:
        url_parse = urlparse(crawler_series['Url'])

        dict_param = parse_qs(url_parse.query)

        crawler_cycle, list_dict = 0, []
        while len(list_dict) < crawler_series['SKU Count']:
            crawler_cycle += 1

            for page in range(crawler_series['Page']):

                crawler_retry = 0
                while True:
                    crawler_retry += 1

                    try:
                        url_request, url_check = f'''https://ecatalog.smpcorp.com/V2/STD/api/part/partsearch?filter={dict_param['part'][0]}&filterType={dict_param['search'][0] if 'search' in dict_param else 'n'}&searchType={dict_param['type'][0] if 'type' in dict_param else 'null'}&imageSize=80&start={page * 96}&limit=96&sort=3&catFilter=-All-&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All-''', crawler_series['Url']

                        resp = requests.get(url_request,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(5, 15))

                        if resp.status_code == 200:
                            list_dict_temp = resp.json()

                            if list_dict_temp:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = =  = = =  = = =

                list_dict.extend(list_dict_temp)

                # = = =  = = =  = = =

                if crawler_remain % 30 == 0:
                    clear_output()
                print(f'''[第{crawler_cycle}轮] - [第{page + 1}页] - [请求{crawler_remain}次] - [{len(list_dict)}/{crawler_series['SKU Count']}] - [剩余{crawler_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_check}\n''')

                # = = =  = = =  = = =

                if len(list_dict) == crawler_series['SKU Count']:
                    break

        # = = =  = = =  = = =

        if not list_dict:
            raise

        # = = =  = = =  = = =

        print('Data Outputting...')
        df_correct = pd.DataFrame([{'No': dict_['rowId'],
                                    'Part Number': dict_['basePart'].strip(),
                                    'Url': f'''https://www.standardbrand.com/en/ecatalog?part={dict_['webBase'].strip()}&type=p&search=s'''} for dict_ in list_dict]).sort_values(by=['No'],
                                                                                                                                                                                  ascending=[True],
                                                                                                                                                                                  ignore_index=True)
        df_correct.to_excel(os.path.join(directory_out, f'''{crawler_series['No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx'''), index=False)
        print('Output Finished !')
        print()
    except:
        crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = =  = = =  = = =

    with semaphore_remain:
        crawler_remain -= 1

# = = =  = = =  = = =

pool.map(crawler, df_input.iterrows())

if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./file/part_number_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

# = = =  = = =  = = =

list_file = [file for file in sorted(list(os.walk(directory_out))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_out, file))]
for file in list_file:
    os.rename(os.path.join(directory_out, file), os.path.join(directory_out, re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)))

# = = =  = = =  = = =

print('Done ~')

Data Loading...
Loading Done !

总数量：1

[第1轮] - [第1页] - [请求1次] - [96/450] - [剩余0条] - [2024/12/11 21:52:12] - 1. https://www.standardbrand.com/en/ecatalog?part=Accelerator%20Pedal%20Sensor%20(APS)&type=p

[第1轮] - [第2页] - [请求1次] - [192/450] - [剩余0条] - [2024/12/11 21:52:17] - 1. https://www.standardbrand.com/en/ecatalog?part=Accelerator%20Pedal%20Sensor%20(APS)&type=p

[第1轮] - [第3页] - [请求1次] - [288/450] - [剩余0条] - [2024/12/11 21:52:21] - 1. https://www.standardbrand.com/en/ecatalog?part=Accelerator%20Pedal%20Sensor%20(APS)&type=p

[第1轮] - [第4页] - [请求1次] - [384/450] - [剩余0条] - [2024/12/11 21:52:27] - 1. https://www.standardbrand.com/en/ecatalog?part=Accelerator%20Pedal%20Sensor%20(APS)&type=p

[第1轮] - [第5页] - [请求1次] - [450/450] - [剩余0条] - [2024/12/11 21:52:29] - 1. https://www.standardbrand.com/en/ecatalog?part=Accelerator%20Pedal%20Sensor%20(APS)&type=p

Data Outputting...
Output Finished !

Done ~
